In [ ]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("sberquad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
import torch
from transformers import pipeline
model_checkpoint = "imangali/mbert-finetuned-sbersquad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = pipeline("question-answering", model=model_checkpoint)

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [ ]:
eval_answers = []
val_answers = []
for i in range(len(dataset['validation'])):
  question, text = dataset['validation'][i]['question'], dataset['validation'][i]['context']

  ans = model(question=question, context=text)
  print(f"ans {ans['answer']}")
  print(f"ans {dataset['validation'][i]['answers']['text'][0]}")

  val_answers.append(dataset['validation'][i]['answers']['text'][0])
  eval_answers.append(ans)

Streaming output truncated to the last 5000 lines.
ans два
ans два брата и две сестры
ans национальные, государственные, классовые и т. п.
ans В последних двух.
ans молодой Юлий Цезарь
ans Юлий Цезарь
ans В. П. Загоровского
ans В. П. Загоровского
ans естественный отбор
ans естественный отбор
ans постоянной сословной комиссией
ans Постоянной сословной комиссией, совместно с советниками герцога
ans к совместному правлению Вильгельма и Людвига
ans К совместному правлению Вильгельма и Людвига
ans восемь
ans восемь
ans Исключительная твёрдость
ans Исключительная твёрдость.
ans энергетическим запасом
ans энергетическим запасом
ans желтушки, большая перламутровка
ans желтушки, большая перламутровка
ans явления сезонного диморфизма и триморфизма
ans Явления сезонного диморфизма и триморфизма
ans в северные районы США и Канады
ans Возвращаются в северные районы США и Канады.
ans до 100 000 особей одновременно
ans до 100 000 особей одновременно
ans в южном направлении
ans Южном
ans Ячмень
ans Яч

In [20]:
evala_answers = []
for i in range(len(dataset['validation'])):
  evala_answers.append(eval_answers[i]['answer'])

In [18]:
vala_answers = []
print(type(dataset['validation'][0]['answers']['text'][0]))
for i in range(len(dataset['validation'])):
  vala_answers.append(dataset['validation'][i]['answers']['text'][0])

<class 'str'>


In [22]:
import collections

num_c = []
num_p = []
num_g = []

for a in range(len(evala_answers)):

  common = collections.Counter(vala_answers[a].split()) & collections.Counter(evala_answers[a].split()) # tokens shared between gold and predicted answers
  num_common = sum(common.values())

  num_pred = len(str(evala_answers[a]).split()) # the number of predicted tokens

  num_gold = len(str(vala_answers[a]).split()) # the number of gold tokens

  num_c.append(num_common)
  num_p.append(num_pred)
  num_g.append(num_gold)

precision = 1.0 * sum(num_c) / sum(num_p) # the num of tokens shared between gold and predicted answers / the num of predicted tokens
recall = 1.0 * sum(num_c) / sum(num_g) # the num of tokens shared between gold and predicted answers / the num of gold tokens
f1_score = (2 * precision * recall) / (precision + recall)

In [23]:
print(f'precision : {precision}')
print(f'recall : {recall}')
print(f'f1_score : {f1_score}')

precision : 0.7377772255699819
recall : 0.6429886302111533
f1_score : 0.6871293430150144
